In [1]:
#http imports
import requests
from bs4 import BeautifulSoup
from datetime import date
import csv
import os
import json
import urllib.request
import pandas as pd
from datetime import date
from datetime import datetime

# pass headers in a dict to the headers parameter to mimic a real human doing the request using a web browser
headers = {
    'accept': '*/*',
    'content-type': 'text/html;charset=UTF-8',
    'content-encoding': 'gzip',
    'origin': 'www.zillow.com',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.71 Safari/537.36'
}

In [2]:
num_clothing = 25 #how many products to scrape
url = "https://us.kowtowclothing.com/collections/all"

#init details
brand = []
product_type = [] #name extracted from url (can be blank)
product_name = []
product_description = []
price = [] #price before sale (can be blank)
sales_price = [] #price customer pays
color = []
material = []
product_url = []
picture_url = []
timestamp = []
gender = [] #website did not have gender specified so all unisex
new_arrival = []

current_date = date.today()

#opening url
response = requests.get(url, headers=headers)

#reading content with BeautifulSoup
soup = BeautifulSoup(response.content, 'html.parser')

products_scraped = 0

product_link_tags = soup.find_all("a", {'class':'grid-product-link-wrapper'})
product_links = []
for tag in product_link_tags:
    product_links.append('https://us.kowtowclothing.com' + tag.get('href'))


In [3]:
page = 1
new_arrival_names = []
while True:
    #new arrivals
    arrivals_url = "https://us.kowtowclothing.com/collections/new-arrivals?page=" + str(page)
    new_arrivals = BeautifulSoup(requests.get(arrivals_url, headers=headers).content, 'html.parser')

    for i in new_arrivals.find_all('p'):
        print(i.get_text())
        if "Sorry, there are no products in this collection" in i.get_text():
            break

            

    new_arrival_links = new_arrivals.find_all("a", {"class":"grid-product-link-wrapper"})
    for tag in new_arrival_links:
        new_arrival_names.append(tag.get("href").split("/")[-1])
    print(page)
    page += 1


    

    
    








      Suggestions: 
      
        
        
          Knitwear
Swimwear
Gift Card
Building Block
Dress
Cardigan



Join the newsletter

Sign up to the newsletter and receive $10 off your first purchase



Enter your email below and we will send you a link to reset your Kowtow account password.



Forgot your password?
To give you a better browsing experience this website uses profiling cookies, including those of third-parties. By continuing to browse you consent to our use of these cookies. Learn more.
Free shipping over $300 USD
Free shipping over $300 USD
Easy returns within 14 days
Easy returns within 14 days
1
['otti-jumper-indigo-check', 'composure-cardigan-primary-red', 'escape-crew-primary-red', 'triangle-skirt-indigo-denim', 'henri-crew-indigo', 'jules-shirt-math-grid', 'lotte-dress-math-grid', 'phoebe-dress-math-grid', 'sailor-jeans-earth-denim', 'triangle-skirt-earth-denim', 'workshop-jacket-earth-denim', 'kiki-top-red', 'kiki-dress-red', 'theory-dress-red', 'composu

KeyboardInterrupt: 

In [ ]:
page_exists = True
page = 1
url = "https://us.kowtowclothing.com/collections/all?page=" + str(page)
while page_exists:
    
    #opening url
    response = requests.get(url, headers=headers)

    #reading content with BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')

    products_scraped = 0

    product_link_tags = soup.find_all("a", {'class':'grid-product-link-wrapper'})
    product_links = []
    for tag in product_link_tags:
        product_links.append('https://us.kowtowclothing.com' + tag.get('href'))
    
    #Check for new arrival
    for link in product_links:
        new = False
        for arrival_name in new_arrival_names:
            if arrival_name == link.split("/")[-1]:
                new = True
                break
        new_arrival.append(new)
        
    count = 0
    while products_scraped < num_clothing and count < len(product_links): #mouse pog
        product_page = BeautifulSoup(requests.get(product_links[count], headers=headers).content, 'html.parser')

        product_div = product_page.find('div', {'itemtype': 'http://schema.org/Product'}) 

        #urls, desc, and brand
        product_metas = product_div.find_all('meta')
        """
        0: product url
        1: description
        2: image url
        3: brand
        ...: not needed
        """
        product_url.append(product_metas[0].get('content'))
        product_description.append(product_metas[1].get('content'))
        picture_url.append(product_metas[2].get('content'))
        brand.append(product_metas[3].get('content'))


        #money and color
        product_misc = product_div.find_all('h2', {'class':'subtitle is-9'})
        product_misc_text = []
        for h2 in product_misc:
            product_misc_text.append(h2.get_text()) #0 and 1 price with \n, 3 is color
        #splitting price
        money_split = product_misc_text[0].split('\n') #0 is '', 1,2 are prices
        price.append(money_split[1])
        sales_price.append(money_split[2])
        color.append(product_misc_text[2])

        #product name
        name = product_div.find('h1', {'itemprop':'name'})
        product_name.append(name.get_text())

        #material
        material_div = product_page.find("div", {'class':"swatch js-swatch"})
        data_tags = material_div['data-product-tags']
        data_tags_split = data_tags.split(",")
        material_info = []
        for tag in data_tags_split:
            if "fabric" in tag or "content" in tag or "process" in tag:
                material_info.append(tag.split(":")[1][:-1])
        material.append(material_info)

        #gender and timestamp
        gender.append('unisex') #website does not have gendered clothing 
        timestamp.append(datetime.now())

        #product type
        product_type.append("none")
        
        count += 1
    page += 1

In [ ]:
#csv output and dataframe
dataframe = pd.DataFrame(list(zip(brand, product_type, product_name, product_description, price, sales_price, color, material, product_url, picture_url, timestamp, gender, new_arrival)), columns = ["Brand", "Product Type", "Product Name", "Description", "Price", "Sales Price", "Color", "Material", "Product URL", "Picture URL", "Timestamp", "Gender", "New Arrival"])


In [ ]:
dataframe